# Descrizione

Una delle ragioni per la quale il k-means non ha buone prestazioni è che usa le distanze spaziali e non ragiona in base alla velocità del mezzo. La velocità cambia a seconda della zona di consegna (urbana, exraurbana, etc ...). Quindi la necessità sta nel riposizionare i punti basandosi sulla velocità della zona specifica, in modo da avere un output del k-means più bilanciato.

Centroide Unico:
Calcola UN SOLO centroide per tutti i punti nell'area urbana specificata

Coordinate: lat ∈ [45.4, 45.55], lon ∈ [9.05, 9.35] (Milano centro)

Tutti i punti urbani vengono "spinti via" da questo singolo punto di riferimento

🎨 Visualizzazione Migliorata:
Stella rossa: Il centroide urbano unico

Rettangolo rosso tratteggiato: La zona urbana di stretching

Frecce arancioni: Mostrano direzione del movimento (sample di punti)

Punti grigi: Posizioni originali per confronto

⚙️ Parametri Configurabili:
lat_range=(45.4, 45.55): Confini latitudine zona urbana

lon_range=(9.05, 9.35): Confini longitudine zona urbana

stretch_factor=2: Moltiplicatore della distanza dal centroide

📊 Logica Semplificata:
Identifica punti nella zona urbana specifica

Calcola centroide medio di questi punti

Per ogni punto urbano: nuova_posizione = centroide + (vecchia_posizione - centroide) × 2

I punti extraurbani rimangono invariati

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random

# -----------------------------
# 1. Carica i dati
# -----------------------------
delivery_data = pd.read_csv('delivery_history.csv')

# -----------------------------
# 2. Filtra i dati (lon compresa tra 5 e 12.5)
# -----------------------------
delivery_data = delivery_data[(delivery_data['lon'] >= 5) & (delivery_data['lon'] <= 12.5)]

# -----------------------------
# 3. Elimina duplicati di location_id (un solo record per cliente)
# -----------------------------
delivery_points = delivery_data[['location_id', 'lat', 'lon']].drop_duplicates(subset='location_id').reset_index(drop=True)

# -----------------------------
# 4. NUOVO: Stretching con SINGOLO CENTROIDE per zona urbana specifica
# -----------------------------
def stretch_urban_zone_single_centroid(points, lat_range=(45.4, 45.55), lon_range=(9.05, 9.35), stretch_factor=1.3):
    """
    Stretcha i punti in una zona urbana specifica usando UN SOLO centroide.
    
    Parametri:
    - points: DataFrame con colonne 'lat', 'lon', 'location_id'
    - lat_range: tupla (min, max) latitudine zona urbana
    - lon_range: tupla (min, max) longitudine zona urbana  
    - stretch_factor: fattore moltiplicativo distanza dal centroide
    
    Ritorna:
    - points_stretched: DataFrame con coordinate modificate
    - centroid: tupla (lat, lon) del centroide
    - num_modified: numero punti modificati
    """
    points = points.copy()
    
    # Identifica punti nella zona urbana specificata
    urban_mask = (
        (points['lat'] >= lat_range[0]) & (points['lat'] <= lat_range[1]) &
        (points['lon'] >= lon_range[0]) & (points['lon'] <= lon_range[1])
    )
    
    urban_points = points.loc[urban_mask]
    num_urban_points = len(urban_points)
    
    if num_urban_points == 0:
        print("⚠️ Nessun punto trovato nell'area urbana specificata")
        return points, None, 0
    
    # Calcola IL CENTROIDE UNICO della zona urbana
    centroid_lat = urban_points['lat'].mean()
    centroid_lon = urban_points['lon'].mean()
    centroid = (centroid_lat, centroid_lon)
    
    # Calcola vettori distanza dal centroide
    delta_lat = urban_points['lat'] - centroid_lat
    delta_lon = urban_points['lon'] - centroid_lon
    
    # Applica stretching: allontana proporzionalmente dal centroide
    points.loc[urban_mask, 'lat'] = centroid_lat + (delta_lat * stretch_factor)
    points.loc[urban_mask, 'lon'] = centroid_lon + (delta_lon * stretch_factor)
    
    # Statistiche
    print(f"🏙️ STRETCHING ZONA URBANA:")
    print(f"   • Zona: lat {lat_range}, lon {lon_range}")
    print(f"   • Centroide urbano: ({centroid_lat:.4f}, {centroid_lon:.4f})")
    print(f"   • Punti nella zona: {num_urban_points}")
    print(f"   • Fattore stretching: {stretch_factor}x")
    print(f"   • Punti modificati: {num_urban_points}")
    
    return points, centroid, num_urban_points

# Applica stretching alla zona urbana specifica
delivery_points_stretched, urban_centroid, num_modified = stretch_urban_zone_single_centroid(
    delivery_points,
    lat_range=(45.4, 45.55),    # Area urbana Milano
    lon_range=(9.05, 9.35),     # Area urbana Milano  
    stretch_factor=2           # 100% più lontani dal centroide
)

# -----------------------------
# 5. Standardizza le coordinate (sui punti stretched)
# -----------------------------
scaler = StandardScaler()
delivery_points_scaled = scaler.fit_transform(delivery_points_stretched[['lat', 'lon']])

# -----------------------------
# 6. Esegui K-Means - MODIFICARE QUI IL NUMERO DI CLUSTER!!!
# -----------------------------
best_k = 50  # puoi modificare questo numero
kmeans = KMeans(n_clusters=best_k, random_state=random.randint(0, 100000), n_init=10)
clusters = kmeans.fit_predict(delivery_points_scaled)

# Aggiungi i cluster al DataFrame stretched
delivery_points_stretched['cluster'] = clusters

# -----------------------------
# 7. Crea un array con location_id per cluster
# -----------------------------
cluster_dict = {}
for c in range(best_k):
    cluster_dict[c+1] = delivery_points_stretched.loc[delivery_points_stretched['cluster'] == c, 'location_id'].tolist()

# Questa è la lista da dare in pasto a Google OR-Tools
list_of_clusters = list(cluster_dict.values())

# Stampa l'output finale
print("\nARRAY FINALE DEI CLUSTER:")
for k, ids in cluster_dict.items():
    print(f"Cluster {k}: {ids}")

# -----------------------------
# 8. Visualizza i cluster con centroide urbano evidenziato
# -----------------------------
plt.figure(figsize=(25,20))

# Genera tanti colori distinti quanti sono i cluster
colors = plt.cm.hsv(np.linspace(0, 1, best_k))

for c in range(best_k):
    subset = delivery_points_stretched[delivery_points_stretched['cluster'] == c]
    plt.scatter(subset['lon'], subset['lat'], s=20, c=[colors[c]], label=f'Cluster {c+1}', 
               edgecolors='k', linewidths=0.5, alpha=0.7)
    
    # Calcola il centroide del cluster per la posizione del numero
    center_lon = subset['lon'].mean()
    center_lat = subset['lat'].mean()
    
    # Inserisci il numero del cluster al centro
    plt.text(center_lon, center_lat, str(c+1), fontsize=12, fontweight='bold', ha='center', va='center',
             color='black', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', boxstyle='round,pad=0.3'))

# EVIDENZIA il centroide urbano e la zona di stretching
if urban_centroid is not None:
    # Centroide urbano principale
    plt.scatter(urban_centroid[1], urban_centroid[0], c='red', s=200, marker='*', 
               edgecolors='black', linewidth=3, label='CENTROIDE URBANO', alpha=1, zorder=10)
    
    # Rettangolo della zona urbana
    from matplotlib.patches import Rectangle
    rect = Rectangle((9.05, 45.4), 0.3, 0.15, linewidth=3, edgecolor='red', 
                    facecolor='red', alpha=0.1, linestyle='--')
    plt.gca().add_patch(rect)
    
    # Linee radiali dal centroide (per mostrare direzione stretching)
    # Sample di alcuni punti urban per mostrare l'effetto
    urban_mask = (
        (delivery_points['lat'] >= 45.4) & (delivery_points['lat'] <= 45.55) &
        (delivery_points['lon'] >= 9.05) & (delivery_points['lon'] <= 9.35)
    )
    urban_sample = delivery_points[urban_mask].sample(min(20, len(delivery_points[urban_mask])))
    urban_stretched_sample = delivery_points_stretched[
        delivery_points_stretched['location_id'].isin(urban_sample['location_id'])
    ]
    
    # Frecce da originale a stretched
    for idx in urban_sample.index:
        original = delivery_points.loc[idx]
        stretched = delivery_points_stretched[
            delivery_points_stretched['location_id'] == original['location_id']
        ].iloc[0]
        
        plt.annotate('', xy=(stretched['lon'], stretched['lat']), 
                    xytext=(original['lon'], original['lat']),
                    arrowprops=dict(arrowstyle='->', color='orange', alpha=0.5, lw=1))

# Mostra alcuni punti originali per confronto
sample_indices = np.random.choice(len(delivery_points), min(300, len(delivery_points)), replace=False)
plt.scatter(delivery_points.iloc[sample_indices]['lon'], 
           delivery_points.iloc[sample_indices]['lat'],
           c='lightgray', s=8, alpha=0.3, label='Posizioni originali (sample)')

plt.xlabel('Longitudine')
plt.ylabel('Latitudine')
plt.title(f'Clustering K-means con Stretching Urbano (Centroide Unico)\n'
          f'K={best_k} | {num_modified} punti modificati nella zona urbana')
plt.legend(loc='best', fontsize='small', ncol=2)
plt.grid(True, alpha=0.3)

# Box informativo
if urban_centroid:
    info_text = (f"🏙️ ZONA URBANA STRETCHATA:\n"
                f"• Lat: [45.4, 45.55]\n"  
                f"• Lon: [9.05, 9.35]\n"
                f"• Centroide: ({urban_centroid[0]:.4f}, {urban_centroid[1]:.4f})\n"
                f"• Punti modificati: {num_modified}\n"
                f"• Fattore stretch: 2x")
    plt.text(0.02, 0.98, info_text, transform=plt.gca().transAxes, 
             verticalalignment='top', 
             bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.8))

plt.show()


In [ ]:
import import_ipynb
import performance_calc as pc

In [ ]:

clusters_performances = pc.calc_clusters_stats(list_of_clusters)



In [ ]:
import pickle

clusters_performances.to_csv('clustering_methods_performances/k-means_stretched_one_centroid_v4(k=50).csv')

with open('cluster_dicts/cluster_dict_k-means_stretched_one_centroid_v4(k=50).pkl', 'wb') as f:
    pickle.dump(cluster_dict, f)

# Prova di un allontanamento inversamente proporzionale alla distanza dal centroide

l'allontanamento è effettuato solo dell'area urbana

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random

# -----------------------------
# 1. Carica i dati
# -----------------------------
delivery_data = pd.read_csv('delivery_history.csv')

# -----------------------------
# 2. Filtra i dati (lon compresa tra 5 e 12.5)
# -----------------------------
delivery_data = delivery_data[(delivery_data['lon'] >= 5) & (delivery_data['lon'] <= 12.5)]

# -----------------------------
# 3. Elimina duplicati di location_id (un solo record per cliente)
# -----------------------------
delivery_points = delivery_data[['location_id', 'lat', 'lon']].drop_duplicates(subset='location_id').reset_index(drop=True)

# -----------------------------
# 4. NUOVO: Stretching INVERSAMENTE PROPORZIONALE alla distanza dal centroide
# -----------------------------
def stretch_inverse_distance_from_centroid(points, lat_range=(45.4, 45.55), lon_range=(9.05, 9.35), 
                                         max_stretch_factor=2.0, min_stretch_factor=1.0):
    """
    Allontana i punti dal centroide in modo inversamente proporzionale alla distanza.
    I punti PIÙ VICINI al centroide vengono allontanati DI PIÙ.
    I punti PIÙ LONTANI dal centroide vengono allontanati DI MENO.
    
    Parametri:
    - points: DataFrame con 'lat', 'lon', 'location_id'
    - lat_range: tupla (min, max) latitudine zona urbana
    - lon_range: tupla (min, max) longitudine zona urbana
    - max_stretch_factor: massimo fattore di allontanamento (per punti vicinissimi al centroide)
    - min_stretch_factor: minimo fattore di allontanamento (per punti lontani dal centroide)
    
    Ritorna:
    - points_stretched: DataFrame con coordinate modificate
    - centroid: tupla (lat, lon) del centroide
    - num_modified: numero punti modificati
    - stretch_stats: statistiche stretching per debugging
    """
    points = points.copy()
    
    # Identifica punti nella zona urbana specificata
    urban_mask = (
        (points['lat'] >= lat_range[0]) & (points['lat'] <= lat_range[1]) &
        (points['lon'] >= lon_range[0]) & (points['lon'] <= lon_range[1])
    )
    
    urban_points = points.loc[urban_mask]
    num_urban_points = len(urban_points)
    
    if num_urban_points == 0:
        print("⚠️ Nessun punto trovato nell'area urbana specificata")
        return points, None, 0, {}
    
    # Calcola IL CENTROIDE UNICO della zona urbana
    centroid_lat = urban_points['lat'].mean()
    centroid_lon = urban_points['lon'].mean()
    centroid = (centroid_lat, centroid_lon)
    
    # Calcola vettori distanza dal centroide
    delta_lat = urban_points['lat'] - centroid_lat
    delta_lon = urban_points['lon'] - centroid_lon
    
    # Calcola distanze euclidee dal centroide
    distances = np.sqrt(delta_lat**2 + delta_lon**2)
    
    # Evita divisione per zero (punti esattamente sul centroide)
    distances_safe = distances.replace(0, 1e-8)  # Valore molto piccolo invece di 0
    
    # Calcola fattore inversamente proporzionale alla distanza
    # Più vicino al centroide = distanza piccola = 1/distanza grande = stretch maggiore
    inverse_distance = 1.0 / distances_safe
    
    # Normalizza i fattori inversi tra 0 e 1
    inverse_distance_normalized = (inverse_distance - inverse_distance.min()) / (inverse_distance.max() - inverse_distance.min())
    
    # Calcola fattori di stretch finali
    # inverse_distance_normalized = 1 (più vicino) -> max_stretch_factor
    # inverse_distance_normalized = 0 (più lontano) -> min_stretch_factor
    stretch_factors = min_stretch_factor + (max_stretch_factor - min_stretch_factor) * inverse_distance_normalized
    
    # Applica stretching: nuova_posizione = centroide + (vecchia_posizione - centroide) * fattore_stretch
    points.loc[urban_mask, 'lat'] = centroid_lat + (delta_lat * stretch_factors)
    points.loc[urban_mask, 'lon'] = centroid_lon + (delta_lon * stretch_factors)
    
    # Statistiche per debugging
    stretch_stats = {
        'min_distance': distances.min(),
        'max_distance': distances.max(),
        'mean_distance': distances.mean(),
        'min_stretch_applied': stretch_factors.min(),
        'max_stretch_applied': stretch_factors.max(),
        'mean_stretch_applied': stretch_factors.mean()
    }
    
    # Output informativo
    print(f"🏙️ STRETCHING INVERSO ALLA DISTANZA:")
    print(f"   • Zona: lat {lat_range}, lon {lon_range}")
    print(f"   • Centroide urbano: ({centroid_lat:.4f}, {centroid_lon:.4f})")
    print(f"   • Punti nella zona: {num_urban_points}")
    print(f"   • Distanza dal centroide: min={stretch_stats['min_distance']:.4f}, max={stretch_stats['max_distance']:.4f}")
    print(f"   • Fattore stretch applicato: min={stretch_stats['min_stretch_applied']:.2f}x, max={stretch_stats['max_stretch_applied']:.2f}x")
    print(f"   • Logica: punti VICINI al centroide → stretch MAGGIORE")
    print(f"   • Logica: punti LONTANI dal centroide → stretch MINORE")
    
    return points, centroid, num_urban_points, stretch_stats

# Applica stretching inverso alla zona urbana specifica
delivery_points_stretched, urban_centroid, num_modified, stretch_info = stretch_inverse_distance_from_centroid(
    delivery_points,
    lat_range=(45.4, 45.55),      # Area urbana Milano
    lon_range=(9.05, 9.35),       # Area urbana Milano  
    max_stretch_factor=3.0,       # Punti vicinissimi: 3x più lontani
    min_stretch_factor=1.1        # Punti lontani: solo 1.1x più lontani
)

# -----------------------------
# 5. Standardizza le coordinate (sui punti stretched)
# -----------------------------
scaler = StandardScaler()
delivery_points_scaled = scaler.fit_transform(delivery_points_stretched[['lat', 'lon']])

# -----------------------------
# 6. Esegui K-Means - MODIFICARE QUI IL NUMERO DI CLUSTER!!!
# -----------------------------
best_k = 50  # puoi modificare questo numero
kmeans = KMeans(n_clusters=best_k, random_state=random.randint(0, 100000), n_init=10)
clusters = kmeans.fit_predict(delivery_points_scaled)

# Aggiungi i cluster al DataFrame stretched
delivery_points_stretched['cluster'] = clusters

# -----------------------------
# 7. Crea un array con location_id per cluster
# -----------------------------
cluster_dict = {}
for c in range(best_k):
    cluster_dict[c+1] = delivery_points_stretched.loc[delivery_points_stretched['cluster'] == c, 'location_id'].tolist()

# Questa è la lista da dare in pasto a Google OR-Tools
list_of_clusters = list(cluster_dict.values())

# Stampa l'output finale
print("\nARRAY FINALE DEI CLUSTER:")
for k, ids in cluster_dict.items():
    print(f"Cluster {k}: {ids}")

# -----------------------------
# 8. Visualizza i cluster con stretching inverso evidenziato
# -----------------------------
plt.figure(figsize=(25,20))

# Genera tanti colori distinti quanti sono i cluster
colors = plt.cm.hsv(np.linspace(0, 1, best_k))

for c in range(best_k):
    subset = delivery_points_stretched[delivery_points_stretched['cluster'] == c]
    plt.scatter(subset['lon'], subset['lat'], s=20, c=[colors[c]], label=f'Cluster {c+1}', 
               edgecolors='k', linewidths=0.5, alpha=0.7)
    
    # Calcola il centroide del cluster per la posizione del numero
    center_lon = subset['lon'].mean()
    center_lat = subset['lat'].mean()
    
    # Inserisci il numero del cluster al centro
    plt.text(center_lon, center_lat, str(c+1), fontsize=12, fontweight='bold', ha='center', va='center',
             color='black', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', boxstyle='round,pad=0.3'))

# EVIDENZIA l'effetto dello stretching inverso
if urban_centroid is not None:
    # Centroide urbano principale
    plt.scatter(urban_centroid[1], urban_centroid[0], c='red', s=300, marker='*', 
               edgecolors='black', linewidth=3, label='CENTROIDE URBANO', alpha=1, zorder=10)
    
    # Rettangolo della zona urbana
    from matplotlib.patches import Rectangle
    rect = Rectangle((9.05, 45.4), 0.3, 0.15, linewidth=3, edgecolor='red', 
                    facecolor='red', alpha=0.1, linestyle='--')
    plt.gca().add_patch(rect)
    
    # Cerchi concentrici per mostrare l'effetto inverso
    for radius in [0.02, 0.05, 0.08]:
        circle = plt.Circle((urban_centroid[1], urban_centroid[0]), radius, 
                          fill=False, color='red', linestyle=':', alpha=0.5, linewidth=2)
        plt.gca().add_patch(circle)
    
    # Frecce da originale a stretched per mostrare l'effetto
    urban_mask = (
        (delivery_points['lat'] >= 45.4) & (delivery_points['lat'] <= 45.55) &
        (delivery_points['lon'] >= 9.05) & (delivery_points['lon'] <= 9.35)
    )
    urban_sample = delivery_points[urban_mask].sample(min(30, len(delivery_points[urban_mask])))
    
    for idx in urban_sample.index:
        original = delivery_points.loc[idx]
        stretched = delivery_points_stretched[
            delivery_points_stretched['location_id'] == original['location_id']
        ].iloc[0]
        
        # Calcola distanza originale dal centroide per colore freccia
        dist_from_center = np.sqrt((original['lat'] - urban_centroid[0])**2 + 
                                  (original['lon'] - urban_centroid[1])**2)
        
        # Colore freccia: rosso (vicino al centro) -> blu (lontano dal centro)
        arrow_color = 'red' if dist_from_center < 0.03 else 'orange' if dist_from_center < 0.06 else 'blue'
        
        plt.annotate('', xy=(stretched['lon'], stretched['lat']), 
                    xytext=(original['lon'], original['lat']),
                    arrowprops=dict(arrowstyle='->', color=arrow_color, alpha=0.7, lw=2))

# Mostra alcuni punti originali per confronto
sample_indices = np.random.choice(len(delivery_points), min(300, len(delivery_points)), replace=False)
plt.scatter(delivery_points.iloc[sample_indices]['lon'], 
           delivery_points.iloc[sample_indices]['lat'],
           c='lightgray', s=8, alpha=0.3, label='Posizioni originali (sample)')

plt.xlabel('Longitudine')
plt.ylabel('Latitudine')
plt.title(f'Clustering K-means con Stretching INVERSO alla Distanza\n'
          f'K={best_k} | {num_modified} punti modificati | Stretch: vicini→MÀX, lontani→min')
plt.legend(loc='best', fontsize='small', ncol=2)
plt.grid(True, alpha=0.3)

# Box informativo dettagliato
if urban_centroid and stretch_info:
    info_text = (f"🎯 STRETCHING INVERSO:\n"
                f"• Centroide: ({urban_centroid[0]:.4f}, {urban_centroid[1]:.4f})\n"
                f"• Punti modificati: {num_modified}\n"
                f"• Stretch min/max: {stretch_info['min_stretch_applied']:.2f}x / {stretch_info['max_stretch_applied']:.2f}x\n"
                f"• Frecce ROSSE: punti vicini (stretch max)\n"
                f"• Frecce BLU: punti lontani (stretch min)\n"
                f"• Cerchi: zone di stretching diverso")
    plt.text(0.02, 0.98, info_text, transform=plt.gca().transAxes, 
             verticalalignment='top', 
             bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9))

plt.show()


In [ ]:
import import_ipynb
import performance_calc as pc

In [ ]:

clusters_performances = pc.calc_clusters_stats(list_of_clusters)


In [ ]:

clusters_performances.to_csv('clustering_methods_performances/k-means_stretched_one_centroid_inversely_proportional_v5(k=50).csv')

with open('cluster_dicts/cluster_dict_k-means_stretched_one_centroid_inversely_proportional_v5(k=50).pkl', 'wb') as f:
    pickle.dump(cluster_dict, f)

# One centroid, inversely proportional stretching but about every delivery point (not just those in the urban area)

 Centroide Fisso Universale:
Calcolato SOLO dai punti nella zona Milano: lat ∈ [45.4, 45.55], lon ∈ [9.05, 9.35]

Applicato a TUTTI i punti del dataset, non solo quelli nella zona di riferimento

Posizione fissa: sempre lo stesso centroide indipendentemente dai punti esterni

🎯 Stretching Inversamente Proporzionale Globale:
Punti VICINI al centroide Milano: stretch 4x (massimo allontanamento)

Punti LONTANI dal centroide Milano: stretch 1x (nessun cambiamento)

Gradiente continuo: da 4x a 1x in base alla distanza dal centroide fisso

🎨 Visualizzazione dell'Effetto Globale:
Stella rossa grande: Il centroide fisso di Milano

Rettangolo rosso: Zona di riferimento per calcolare il centroide

Cerchi concentrici: Zone di stretching decrescente dal centro

Frecce colorate:

ROSSE (spesse): Punti vicini con stretch 4x

ARANCIONI (medie): Punti medi con stretch intermedio

VERDI (sottili): Punti lontani con stretch 1x

📊 Logica Matematica:
Centroide fisso: Media lat/lon dei punti Milano

Distanze globali: Da TUTTI i punti al centroide fisso

Fattore inverso: 1/distanza normalizzato 0-1

Stretch finale: 1 + 3 × fattore_inverso (da 1x a 4x)

Applicazione: nuovo = centroide + (vecchio-centroide) × stretch

Questo approccio crea un effetto radiale globale dove Milano è il "centro di espansione" e tutti i punti vengono influenzati proporzionalmente alla loro distanza da questo centro fisso, simulando l'effetto del traffico urbano che si propaga dal centro città verso l'esterno!

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random
import import_ipynb
import performance_calc as pc

# -----------------------------
# 4. NUOVO: Stretching GLOBALE da centroide fisso (4x → 1x)
# -----------------------------
def stretch_all_points_from_fixed_centroid(points, ref_lat_range=(45.4, 45.55), ref_lon_range=(9.05, 9.35), 
                                          max_stretch=4.0, min_stretch=1.0):
    """
    Allontana TUTTI i punti del dataset da un centroide fisso calcolato dalla zona di riferimento.
    Lo stretching è inversamente proporzionale alla distanza:
    - Punti VICINI al centroide: stretch MASSIMO (4x)
    - Punti LONTANI dal centroide: stretch MINIMO (1x)
    
    Parametri:
    - points: DataFrame con 'lat', 'lon', 'location_id'
    - ref_lat_range: zona di riferimento per calcolare il centroide
    - ref_lon_range: zona di riferimento per calcolare il centroide
    - max_stretch: fattore massimo (punti vicini al centroide)
    - min_stretch: fattore minimo (punti lontani dal centroide)
    """
    points = points.copy()
    
    # Identifica punti nella zona di riferimento per calcolare il centroide
    ref_zone_mask = (
        (points['lat'] >= ref_lat_range[0]) & (points['lat'] <= ref_lat_range[1]) &
        (points['lon'] >= ref_lon_range[0]) & (points['lon'] <= ref_lon_range[1])
    )
    
    ref_zone_points = points.loc[ref_zone_mask]
    
    if len(ref_zone_points) == 0:
        print("⚠️ Nessun punto nella zona di riferimento per calcolare il centroide")
        return points, None, 0
    
    # Calcola IL CENTROIDE FISSO dalla zona di riferimento
    centroid_lat = ref_zone_points['lat'].mean()
    centroid_lon = ref_zone_points['lon'].mean()
    centroid = (centroid_lat, centroid_lon)
    
    # Calcola vettori distanza dal centroide per TUTTI i punti
    delta_lat = points['lat'] - centroid_lat
    delta_lon = points['lon'] - centroid_lon
    
    # Calcola distanze euclidee dal centroide per TUTTI i punti
    distances = np.sqrt(delta_lat**2 + delta_lon**2)
    
    # Evita divisione per zero
    distances_safe = distances.replace(0, 1e-8)
    
    # Calcola fattore inversamente proporzionale per TUTTI i punti
    inverse_distance = 1.0 / distances_safe
    
    # Normalizza tra 0 e 1
    inverse_distance_normalized = (inverse_distance - inverse_distance.min()) / (inverse_distance.max() - inverse_distance.min())
    
    # Calcola fattori di stretch finali per TUTTI i punti
    # inverse_distance_normalized = 1 (più vicino) -> max_stretch (4x)
    # inverse_distance_normalized = 0 (più lontano) -> min_stretch (1x)
    stretch_factors = min_stretch + (max_stretch - min_stretch) * inverse_distance_normalized
    
    # Applica stretching a TUTTI i punti
    points['lat'] = centroid_lat + (delta_lat * stretch_factors)
    points['lon'] = centroid_lon + (delta_lon * stretch_factors)
    
    # Statistiche
    num_points_modified = len(points)
    
    print(f"🌍 STRETCHING GLOBALE DA CENTROIDE FISSO:")
    print(f"   • Zona di riferimento centroide: lat {ref_lat_range}, lon {ref_lon_range}")
    print(f"   • Centroide fisso: ({centroid_lat:.4f}, {centroid_lon:.4f})")
    print(f"   • Punti nella zona di riferimento: {len(ref_zone_points)}")
    print(f"   • TUTTI i punti modificati: {num_points_modified}")
    print(f"   • Range stretching: {min_stretch}x (lontani) → {max_stretch}x (vicini)")
    print(f"   • Distanza min/max dal centroide: {distances.min():.4f} / {distances.max():.4f}")
    print(f"   • Stretch min/max applicato: {stretch_factors.min():.2f}x / {stretch_factors.max():.2f}x")
    
    return points, centroid, num_points_modified

# Applica stretching globale con centroide fisso
delivery_points_stretched, fixed_centroid, num_modified = stretch_all_points_from_fixed_centroid(
    pc.delivery_points_AS,
    ref_lat_range=(45.4, 45.55),    # Zona Milano centro per calcolare centroide
    ref_lon_range=(9.05, 9.35),     # Zona Milano centro per calcolare centroide
    max_stretch=4.0,                # Punti vicini: 4x più lontani
    min_stretch=1.0                 # Punti lontani: nessun cambiamento
)

# -----------------------------
# 5. Standardizza le coordinate (sui punti stretched)
# -----------------------------
scaler = StandardScaler()
delivery_points_scaled = scaler.fit_transform(delivery_points_stretched[['lat', 'lon']])

# -----------------------------
# 6. Esegui K-Means - MODIFICARE QUI IL NUMERO DI CLUSTER!!!
# -----------------------------
best_k = 50  # puoi modificare questo numero
# kmeans = KMeans(n_clusters=best_k, random_state=random.randint(0, 100000), n_init=10)
kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(delivery_points_scaled)

# Aggiungi i cluster al DataFrame stretched
delivery_points_stretched['cluster'] = clusters

# -----------------------------
# 7. Crea un array con location_id per cluster
# -----------------------------
cluster_dict = {}
for c in range(best_k):
    cluster_dict[c+1] = delivery_points_stretched.loc[delivery_points_stretched['cluster'] == c, 'location_id'].tolist()

# Questa è la lista da dare in pasto a Google OR-Tools
list_of_clusters = list(cluster_dict.values())

# Stampa l'output finale
print("\nARRAY FINALE DEI CLUSTER:")
for k, ids in cluster_dict.items():
    print(f"Cluster {k}: {ids}")

# -----------------------------
# 8. Visualizza risultati con stretching globale
# -----------------------------
plt.figure(figsize=(25,20))

# Genera colori per i cluster
colors = plt.cm.hsv(np.linspace(0, 1, best_k))

for c in range(best_k):
    subset = delivery_points_stretched[delivery_points_stretched['cluster'] == c]
    plt.scatter(subset['lon'], subset['lat'], s=20, c=[colors[c]], label=f'Cluster {c+1}', 
               edgecolors='k', linewidths=0.5, alpha=0.7)
    
    # Numero del cluster al centroide
    center_lon = subset['lon'].mean()
    center_lat = subset['lat'].mean()
    plt.text(center_lon, center_lat, str(c+1), fontsize=12, fontweight='bold', ha='center', va='center',
             color='black', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', boxstyle='round,pad=0.3'))

# EVIDENZIA il centroide fisso e l'effetto globale
if fixed_centroid is not None:
    # Centroide fisso (calcolato dalla zona Milano)
    plt.scatter(fixed_centroid[1], fixed_centroid[0], c='red', s=400, marker='*', 
               edgecolors='black', linewidth=4, label='CENTROIDE FISSO (Milano)', alpha=1, zorder=15)
    
    # Rettangolo zona di riferimento
    from matplotlib.patches import Rectangle
    rect = Rectangle((9.05, 45.4), 0.3, 0.15, linewidth=4, edgecolor='red', 
                    facecolor='red', alpha=0.15, linestyle='--', 
                    label='Zona riferimento centroide')
    plt.gca().add_patch(rect)
    
    # Cerchi concentrici per mostrare zone di stretching diverso
    for radius, alpha in [(0.1, 0.3), (0.3, 0.2), (0.6, 0.1), (1.0, 0.05)]:
        circle = plt.Circle((fixed_centroid[1], fixed_centroid[0]), radius, 
                          fill=False, color='red', linestyle=':', alpha=alpha, linewidth=3)
        plt.gca().add_patch(circle)
    
    # Frecce per mostrare l'effetto dello stretching (sample)
    sample_indices = np.random.choice(len(pc.delivery_points_AS), min(50, len(pc.delivery_points_AS)), replace=False)
    
    for idx in sample_indices:
        original = pc.delivery_points_AS.iloc[idx]
        stretched = delivery_points_stretched.iloc[idx]
        
        # Calcola distanza originale dal centroide fisso per colore freccia
        dist_from_center = np.sqrt((original['lat'] - fixed_centroid[0])**2 + 
                                  (original['lon'] - fixed_centroid[1])**2)
        
        # Gradiente colore: rosso (vicino) -> giallo (medio) -> verde (lontano)
        if dist_from_center < 0.2:
            arrow_color = 'red'
            arrow_width = 3
        elif dist_from_center < 0.5:
            arrow_color = 'orange' 
            arrow_width = 2
        else:
            arrow_color = 'green'
            arrow_width = 1
        
        plt.annotate('', xy=(stretched['lon'], stretched['lat']), 
                    xytext=(original['lon'], original['lat']),
                    arrowprops=dict(arrowstyle='->', color=arrow_color, alpha=0.6, lw=arrow_width))

# Mostra alcuni punti originali per confronto (molto trasparenti)
sample_original = np.random.choice(len(pc.delivery_points_AS), min(200, len(pc.delivery_points_AS)), replace=False)
plt.scatter(pc.delivery_points_AS.iloc[sample_original]['lon'], 
           pc.delivery_points_AS.iloc[sample_original]['lat'],
           c='lightgray', s=3, alpha=0.2, label='Posizioni originali (sample)')

plt.xlabel('Longitudine')
plt.ylabel('Latitudine')
plt.title(f'Clustering K-means con Stretching GLOBALE da Centroide Fisso\n'
          f'K={best_k} | TUTTI i {num_modified} punti modificati | Stretch: 4x→1x dal centroide Milano')
plt.legend(loc='best', fontsize='small', ncol=2)
plt.grid(True, alpha=0.3)

# Box informativo dettagliato
if fixed_centroid:
    info_text = (f"🎯 STRETCHING GLOBALE:\n"
                f"• Centroide fisso: ({fixed_centroid[0]:.4f}, {fixed_centroid[1]:.4f})\n"
                f"• Zona centroide: Milano [45.4-45.55, 9.05-9.35]\n"
                f"• TUTTI i punti modificati: {num_modified}\n"
                f"• Stretch: 4x (vicini) → 1x (lontani)\n"
                f"• Frecce ROSSE: punti vicini (stretch 4x)\n"
                f"• Frecce VERDI: punti lontani (stretch 1x)\n"
                f"• Cerchi: zone di stretching decrescente")
    plt.text(0.02, 0.98, info_text, transform=plt.gca().transAxes, 
             verticalalignment='top', 
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.9))

plt.show()

In [ ]:
clusters_performances = pc.calc_clusters_stats_AS(list_of_clusters)

In [ ]:
import pickle
clusters_performances.to_csv('clustering_methods_performances/k-means_stretched_one_centroid_inversely_proportional_v6(k=50)_AS.csv')

with open('cluster_dicts/cluster_dict_k-means_stretched_one_centroid_inversely_proportional_v6(k=50)_AS.pkl', 'wb') as f:
    pickle.dump(cluster_dict, f)

Ripeto l'algoritmo solo per runnarlo due volte insieme senza toccarlo

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random

# -----------------------------
# 4. NUOVO: Stretching GLOBALE da centroide fisso (4x → 1x)
# -----------------------------
def stretch_all_points_from_fixed_centroid(points, ref_lat_range=(45.4, 45.55), ref_lon_range=(9.05, 9.35), 
                                          max_stretch=4.0, min_stretch=1.0):
    """
    Allontana TUTTI i punti del dataset da un centroide fisso calcolato dalla zona di riferimento.
    Lo stretching è inversamente proporzionale alla distanza:
    - Punti VICINI al centroide: stretch MASSIMO (4x)
    - Punti LONTANI dal centroide: stretch MINIMO (1x)
    
    Parametri:
    - points: DataFrame con 'lat', 'lon', 'location_id'
    - ref_lat_range: zona di riferimento per calcolare il centroide
    - ref_lon_range: zona di riferimento per calcolare il centroide
    - max_stretch: fattore massimo (punti vicini al centroide)
    - min_stretch: fattore minimo (punti lontani dal centroide)
    """
    points = points.copy()
    
    # Identifica punti nella zona di riferimento per calcolare il centroide
    ref_zone_mask = (
        (points['lat'] >= ref_lat_range[0]) & (points['lat'] <= ref_lat_range[1]) &
        (points['lon'] >= ref_lon_range[0]) & (points['lon'] <= ref_lon_range[1])
    )
    
    ref_zone_points = points.loc[ref_zone_mask]
    
    if len(ref_zone_points) == 0:
        print("⚠️ Nessun punto nella zona di riferimento per calcolare il centroide")
        return points, None, 0
    
    # Calcola IL CENTROIDE FISSO dalla zona di riferimento
    centroid_lat = ref_zone_points['lat'].mean()
    centroid_lon = ref_zone_points['lon'].mean()
    centroid = (centroid_lat, centroid_lon)
    
    # Calcola vettori distanza dal centroide per TUTTI i punti
    delta_lat = points['lat'] - centroid_lat
    delta_lon = points['lon'] - centroid_lon
    
    # Calcola distanze euclidee dal centroide per TUTTI i punti
    distances = np.sqrt(delta_lat**2 + delta_lon**2)
    
    # Evita divisione per zero
    distances_safe = distances.replace(0, 1e-8)
    
    # Calcola fattore inversamente proporzionale per TUTTI i punti
    inverse_distance = 1.0 / distances_safe
    
    # Normalizza tra 0 e 1
    inverse_distance_normalized = (inverse_distance - inverse_distance.min()) / (inverse_distance.max() - inverse_distance.min())
    
    # Calcola fattori di stretch finali per TUTTI i punti
    # inverse_distance_normalized = 1 (più vicino) -> max_stretch (4x)
    # inverse_distance_normalized = 0 (più lontano) -> min_stretch (1x)
    stretch_factors = min_stretch + (max_stretch - min_stretch) * inverse_distance_normalized
    
    # Applica stretching a TUTTI i punti
    points['lat'] = centroid_lat + (delta_lat * stretch_factors)
    points['lon'] = centroid_lon + (delta_lon * stretch_factors)
    
    # Statistiche
    num_points_modified = len(points)
    
    print(f"🌍 STRETCHING GLOBALE DA CENTROIDE FISSO:")
    print(f"   • Zona di riferimento centroide: lat {ref_lat_range}, lon {ref_lon_range}")
    print(f"   • Centroide fisso: ({centroid_lat:.4f}, {centroid_lon:.4f})")
    print(f"   • Punti nella zona di riferimento: {len(ref_zone_points)}")
    print(f"   • TUTTI i punti modificati: {num_points_modified}")
    print(f"   • Range stretching: {min_stretch}x (lontani) → {max_stretch}x (vicini)")
    print(f"   • Distanza min/max dal centroide: {distances.min():.4f} / {distances.max():.4f}")
    print(f"   • Stretch min/max applicato: {stretch_factors.min():.2f}x / {stretch_factors.max():.2f}x")
    
    return points, centroid, num_points_modified

# Applica stretching globale con centroide fisso
delivery_points_stretched, fixed_centroid, num_modified = stretch_all_points_from_fixed_centroid(
    pc.delivery_points_ON,
    ref_lat_range=(45.4, 45.55),    # Zona Milano centro per calcolare centroide
    ref_lon_range=(9.05, 9.35),     # Zona Milano centro per calcolare centroide
    max_stretch=4.0,                # Punti vicini: 4x più lontani
    min_stretch=1.0                 # Punti lontani: nessun cambiamento
)

# -----------------------------
# 5. Standardizza le coordinate (sui punti stretched)
# -----------------------------
scaler = StandardScaler()
delivery_points_scaled = scaler.fit_transform(delivery_points_stretched[['lat', 'lon']])

# -----------------------------
# 6. Esegui K-Means - MODIFICARE QUI IL NUMERO DI CLUSTER!!!
# -----------------------------
best_k = 50  # puoi modificare questo numero
kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(delivery_points_scaled)

# Aggiungi i cluster al DataFrame stretched
delivery_points_stretched['cluster'] = clusters

# -----------------------------
# 7. Crea un array con location_id per cluster
# -----------------------------
cluster_dict = {}
for c in range(best_k):
    cluster_dict[c+1] = delivery_points_stretched.loc[delivery_points_stretched['cluster'] == c, 'location_id'].tolist()

# Questa è la lista da dare in pasto a Google OR-Tools
list_of_clusters = list(cluster_dict.values())

# Stampa l'output finale
print("\nARRAY FINALE DEI CLUSTER:")
for k, ids in cluster_dict.items():
    print(f"Cluster {k}: {ids}")

# -----------------------------
# 8. Visualizza risultati con stretching globale
# -----------------------------
plt.figure(figsize=(25,20))

# Genera colori per i cluster
colors = plt.cm.hsv(np.linspace(0, 1, best_k))

for c in range(best_k):
    subset = delivery_points_stretched[delivery_points_stretched['cluster'] == c]
    plt.scatter(subset['lon'], subset['lat'], s=20, c=[colors[c]], label=f'Cluster {c+1}', 
               edgecolors='k', linewidths=0.5, alpha=0.7)
    
    # Numero del cluster al centroide
    center_lon = subset['lon'].mean()
    center_lat = subset['lat'].mean()
    plt.text(center_lon, center_lat, str(c+1), fontsize=12, fontweight='bold', ha='center', va='center',
             color='black', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', boxstyle='round,pad=0.3'))

# EVIDENZIA il centroide fisso e l'effetto globale
if fixed_centroid is not None:
    # Centroide fisso (calcolato dalla zona Milano)
    plt.scatter(fixed_centroid[1], fixed_centroid[0], c='red', s=400, marker='*', 
               edgecolors='black', linewidth=4, label='CENTROIDE FISSO (Milano)', alpha=1, zorder=15)
    
    # Rettangolo zona di riferimento
    from matplotlib.patches import Rectangle
    rect = Rectangle((9.05, 45.4), 0.3, 0.15, linewidth=4, edgecolor='red', 
                    facecolor='red', alpha=0.15, linestyle='--', 
                    label='Zona riferimento centroide')
    plt.gca().add_patch(rect)
    
    # Cerchi concentrici per mostrare zone di stretching diverso
    for radius, alpha in [(0.1, 0.3), (0.3, 0.2), (0.6, 0.1), (1.0, 0.05)]:
        circle = plt.Circle((fixed_centroid[1], fixed_centroid[0]), radius, 
                          fill=False, color='red', linestyle=':', alpha=alpha, linewidth=3)
        plt.gca().add_patch(circle)
    
    # Frecce per mostrare l'effetto dello stretching (sample)
    sample_indices = np.random.choice(len(pc.delivery_points_ON), min(50, len(pc.delivery_points_ON)), replace=False)
    
    for idx in sample_indices:
        original = pc.delivery_points_ON.iloc[idx]
        stretched = delivery_points_stretched.iloc[idx]
        
        # Calcola distanza originale dal centroide fisso per colore freccia
        dist_from_center = np.sqrt((original['lat'] - fixed_centroid[0])**2 + 
                                  (original['lon'] - fixed_centroid[1])**2)
        
        # Gradiente colore: rosso (vicino) -> giallo (medio) -> verde (lontano)
        if dist_from_center < 0.2:
            arrow_color = 'red'
            arrow_width = 3
        elif dist_from_center < 0.5:
            arrow_color = 'orange' 
            arrow_width = 2
        else:
            arrow_color = 'green'
            arrow_width = 1
        
        plt.annotate('', xy=(stretched['lon'], stretched['lat']), 
                    xytext=(original['lon'], original['lat']),
                    arrowprops=dict(arrowstyle='->', color=arrow_color, alpha=0.6, lw=arrow_width))

# Mostra alcuni punti originali per confronto (molto trasparenti)
sample_original = np.random.choice(len(pc.delivery_points_ON), min(200, len(pc.delivery_points_ON)), replace=False)
plt.scatter(pc.delivery_points_ON.iloc[sample_original]['lon'], 
           pc.delivery_points_ON.iloc[sample_original]['lat'],
           c='lightgray', s=3, alpha=0.2, label='Posizioni originali (sample)')

plt.xlabel('Longitudine')
plt.ylabel('Latitudine')
plt.title(f'Clustering K-means con Stretching GLOBALE da Centroide Fisso\n'
          f'K={best_k} | TUTTI i {num_modified} punti modificati | Stretch: 4x→1x dal centroide Milano')
plt.legend(loc='best', fontsize='small', ncol=2)
plt.grid(True, alpha=0.3)

# Box informativo dettagliato
if fixed_centroid:
    info_text = (f"🎯 STRETCHING GLOBALE:\n"
                f"• Centroide fisso: ({fixed_centroid[0]:.4f}, {fixed_centroid[1]:.4f})\n"
                f"• Zona centroide: Milano [45.4-45.55, 9.05-9.35]\n"
                f"• TUTTI i punti modificati: {num_modified}\n"
                f"• Stretch: 4x (vicini) → 1x (lontani)\n"
                f"• Frecce ROSSE: punti vicini (stretch 4x)\n"
                f"• Frecce VERDI: punti lontani (stretch 1x)\n"
                f"• Cerchi: zone di stretching decrescente")
    plt.text(0.02, 0.98, info_text, transform=plt.gca().transAxes, 
             verticalalignment='top', 
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.9))

plt.show()

clusters_performances = pc.calc_clusters_stats_ON(list_of_clusters)
clusters_performances.to_csv('clustering_methods_performances/k-means_stretched_one_centroid_inversely_proportional_v6(k=50)_ON.csv')

with open('cluster_dicts/cluster_dict_k-means_stretched_one_centroid_inversely_proportional_v6(k=50)_ON.pkl', 'wb') as f:
    pickle.dump(cluster_dict, f)